In [1]:
import pandas as pd
books = pd.read_csv("books.csv", sep=";",on_bad_lines="skip",encoding='latin-1')
ratings = pd.read_csv("ratings.csv", sep=";",on_bad_lines="skip",encoding='latin-1')

C:\Users\lol_a\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3505: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
'''
In the original dataset some ISBN were registered in ratings 
but not in books.
The variable names were changed by convenience.
After cleaning ratings I will use MySQL to extract the cols.
'''
books['isbn'] = books['isbn'].str.strip().str.lower()
ratings['isbn'] = ratings['isbn'].str.strip().str.lower()
missing = set(ratings['isbn']) - set(books['isbn'])
print(f'ISBN ratings not in books: {len(missing)}')
print(list(missing)[:10])  

ISBN ratings not in books: 70372
['1561342017', '8833906019', '0781400309', '9782253152866', '0340682086', '2.02.055078.4', '884620090x', '0586058494', '0345249372150', '2748007514']


In [3]:
# Cleaning ratings
valid_ratings = ratings[ratings['isbn'].isin(books['isbn'])]
print(f'Original records: {len(ratings)}')
print(f'Valid records: {len(valid_ratings)}')
valid_ratings.to_csv('ratings_clean.csv', sep=';', index=False)

Original records: 1149780
Valid records: 1031139


In [ ]:
'''
Loaded the 3 files into a MySQL database using ratings_clean instead of ratings.
After loading this query the dataset of interes is obtained.
'''
SELECT b.title, b.author, b.year, b.isbn, AVG(r.rating) AS rating FROM ratings AS r
JOIN users AS u ON r.userid = u.userid
JOIN books AS b ON r.isbn = b.isbn
WHERE r.rating > 0
AND b.year > 0
AND b.title IS NOT NULL
AND b.author IS NOT NULL
AND u.userid IS NOT NULL
AND b.isbn IS NOT NULL
GROUP BY r.isbn
ORDER BY rating DESC
INTO OUTFILE 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/librarydb_.csv'
FIELDS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\r\n';